In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, model_selection, naive_bayes, metrics

In [90]:
data = pd.read_csv('spam.csv', encoding='latin-1')

In [91]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [92]:
f = feature_extraction.text.CountVectorizer(stop_words = 'english')
X = f.fit_transform(data["v2"])
np.shape(X)

(5572, 8404)

In [93]:
data["v1"]=data["v1"].map({'spam':1,'ham':0})
X_latih, X_uji, y_latih, y_uji = model_selection.train_test_split(X, data['v1'], test_size=0.3, random_state=42)
print([np.shape(X_latih), np.shape(X_uji)])

[(3900, 8404), (1672, 8404)]


In [94]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_latih, y_latih)
    score_train[count] = bayes.score(X_latih, y_latih)
    score_test[count]= bayes.score(X_uji, y_uji)
    recall_test[count] = metrics.recall_score(y_uji, bayes.predict(X_uji))
    precision_test[count] = metrics.precision_score(y_uji, bayes.predict(X_uji))
    count = count + 1

In [95]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns =
             ['alpha', 'Akurasi latih', 'Akurasi uji', 'uji Recall', 'uji Presisi'])
models.head(n=10)

,alpha,Akurasi latih,Akurasi uji,uji Recall,uji Presisi
0,0.00001,0.998205,0.972488,0.913242,0.881057
1,0.11001,0.996923,0.975478,0.931507,0.886957
2,0.22001,0.997179,0.977273,0.931507,0.898678
3,0.33001,0.997179,0.976077,0.931507,0.890830
4,0.44001,0.996667,0.977273,0.931507,0.898678
5,0.55001,0.996154,0.976675,0.931507,0.894737
6,0.66001,0.995897,0.976675,0.931507,0.894737
7,0.77001,0.995897,0.976675,0.926941,0.898230
8,0.88001,0.995897,0.977273,0.922374,0.905830
9,0.99001,0.995385,0.977273,0.922374,0.905830


In [96]:
m_confusion_test = metrics.confusion_matrix(y_uji, bayes.predict(X_uji))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,1453,0
Actual 1,58,161
